In [1]:
import numpy as np
import pandas as pd
import keras
import eng_to_ipa as ipa

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Multiple_Forms.csv', index_col = 0)
df.head()

,Author,Poem,Form
0,William Shakespeare,"From fairest creatures we desire increase,\r\n...",Sonnet
1,William Shakespeare,"When forty winters shall besiege thy brow,\r\n...",Sonnet
2,William Shakespeare,Look in thy glass and tell the face thou viewe...,Sonnet
3,William Shakespeare,"Unthrifty loveliness, why dost thou spend\r\n ...",Sonnet
4,William Shakespeare,"Those hours, that with gentle work did frame\r...",Sonnet


In [3]:
poems = df['Poem']
poems.head(20)

0     From fairest creatures we desire increase,\r\n...
1     When forty winters shall besiege thy brow,\r\n...
2     Look in thy glass and tell the face thou viewe...
3     Unthrifty loveliness, why dost thou spend\r\n ...
4     Those hours, that with gentle work did frame\r...
5     Then let not winter's ragged hand deface,\r\n ...
6     Lo! in the orient when the gracious light\r\n ...
7     Music to hear, why hear'st thou music sadly?\r...
8     Is it for fear to wet a widow's eye,\r\n  That...
9     For shame! deny that thou bear'st love to any,...
10    As fast as thou shalt wane, so fast thou grow'...
11    When I do count the clock that tells the time,...
12    O! that you were your self; but, love you are\...
13    Not from the stars do I my judgement pluck;\r\...
14    When I consider every thing that grows\r\n  Ho...
15    But wherefore do not you a mightier way\r\n  M...
16    Who will believe my verse in time to come,\r\n...
17    Shall I compare thee to a summer's day?\r\

In [4]:
text = ''
for poem in poems:
    text += str(poem) + '\n'
len(text)

780446

In [5]:
print(text)

From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou, contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thyself thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
        Pity the world, or else this glutton be,
        To eat the world's due, by the grave and thee.
When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tatter'd weed of small worth held:
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say, within thine own deep sunken eyes,
  Were an all-eating shame, and thr

In [6]:
word_list = text.split()
len(word_list)

139668

In [7]:
maxlen = 6                                                            
step = 1                                                               

sentences = []                                                         

next_words = []                                                        

for i in range(0, len(word_list) - maxlen, step):
    sentences.append(word_list[i: i + maxlen])
    next_words.append(word_list[i + maxlen])

print('Number of sequences:', len(sentences))

words = sorted(list(set(word_list)))                                        
print('Unique words:', len(words))
word_indices = dict((word, words.index(word)) for word in words)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)      
y = np.zeros((len(sentences), len(words)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, word in enumerate(sentence):                                
        x[i, t, word_indices[word]] = 1                                
    y[i, word_indices[next_words[i]]] = 1 

Number of sequences: 139662
Unique words: 25686
Vectorization...


In [8]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(words))))
model.add(layers.Dense(len(words), activation='softmax'))

In [9]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [10]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
import random
import sys

generated_text = ''
count = 0

for epoch in range(1, 21):
    generated_text = ''
    generated_list = []
    print('\nepoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(word_list) - maxlen - 1)         
    generated_list = word_list[start_index: start_index + maxlen]
#     generated_list = ['The', 'cat', 'in', 'hat']
    for word in generated_list:
        generated_text += word + ' '
    generated_text =  generated_text.strip()
    print('--- Generating with seed: "' + generated_text + '"')
    for num in ipa.syllable_count(generated_text):
        count += num
    for temperature in [0.2, 0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(words)))             
            for t, word in enumerate(generated_list):               
                sampled[0, t, word_indices[word]] = 1.              

            preds = model.predict(sampled, verbose=0)[0]            
            next_index = sample(preds, temperature)                 
            next_word = words[next_index]                           

            generated_list.append(next_word)
            generated_list = generated_list[1:]

            sys.stdout.write(' ' + next_word)
            if ipa.syllable_count(next_word) == 0:
                if len(next_word) >= 9:
                    count += 3
                if len(next_word) >= 6:
                    count += 2
                else:
                    count += 1
            else:
                count += ipa.syllable_count(next_word)

            if count >= 10:
                sys.stdout.write('\n')
                count = 0


epoch 1
Epoch 1/1
139662/139662 [==============================] - 743s 5ms/step - loss: 7.7402
--- Generating with seed: "then to tear my breast! “The"
------ temperature: 0.2
then to tear my breast! “The I have the love of
 the world of the heart of the world of the
 breath of the breath of the breath of the old
 And the man of the wind of the world of
 a man of a man And the man of the
 wind of the world of the heart of the world
 of a heart of a man of the great And
 the man of the wind of the world of the
 end of the breath of the heart of the wind
 of the heart of the wind of the sake And
 the man of the little of the man of
 the heart of the great of a heart of the
 breath of the heart of the sweet of a breath
 of the world of the heart of a little
 And I have a man of a man of a
 heart of the breath of the heart of the world
 of a thing And the man of the wind of
 a man And the man of the wind And the
 man of the world of the old And the man
 of the wind of the heart of the wo

 is a I am not a I am I am
 not say I am not a My heart that I
 am not a My heart is a man who never
 a man man who looked And saw a man a
 man a man who a man had a thing he
 could been a thing I am I am not ;
 And I am not a I am not ; And
 I am I am not a My love is a
 I am not a I am not a My heart
 is a man he had a thing he could not
 a a man doth a a man a man who
 looked And saw a man a man he could not
 a a man that never a man he could
 not a a man had been a thing he had
 been a thing he could not touch Nor such a
 love in thy love is a I am not a
 I am not a I am not a I am
 not ; And I am not a I am not
 a I am not a My heart is a most
 Whose Thy love is a man is a thing he
 could not a a thing I see my love have
 love I have a I am not a My little
 heart of a love is a I am not a
 I am not a My love is a I am
 not a I am not to thee My love is
------ temperature: 0.5
not how, used more to spill a That I am in the night Is a it
 had a a man thing he to a man had
 a use a m

 the of of a of of a of the of
 this of of the of of a of of contrast
 and like a of of a of the Which of
 the of of my of But we are the of
 of of and of a of are a of the
 a of the of of of we and dignity
 of the will of the of of the of of
 of of a of a and of the a of
 of and of the of of fere of the cruiser's
 of the case; of the and of the a of
 my of And and of their a separable of
 the springs, of the of the of of of and
 of and their And of the of of of of
 of and of in a of a of the so
 of of that of of of of or of In
 of a of a of a Of a of the
 of of and of a a man of that And
 at the of of their and sheathing of the
 of of through. of a of of monument.
 and a of a of course, and arm, and a
 and of the a of Love of a and a
 of old or and the a of a of the
 Thou of my my and will and no of the
 of my of and of a a of His will
 will not of the of that of of of a
 of a time of And that will I of of
 my thoughts and will my of my of a to
 their a and of of a are of the of
 the 

 a sad sad he. he. and a Only a
 and green and and and a -- and and at
 a friendly at once is more than and at
 and and and a and that has a and ring
 of a You has to a and of a life
 and a good and and a good and and a
 smile, and and a Our green and has a a
 and has a has o' has and has has no
 of a and has has has a has and has
 before has has no need of a -- and
 at a life's last look there has a a man,
 and a white man Is a at and and and
 a Our God has a and and a man has
 has a a wretched man, and a man, and
 Whose that's that's and hold a a man, and and
 at each and a man has has a a and
 white has a and a whose life has has no
 more -- and write is a and white and a
------ temperature: 1.0
wine upon a sponge Was the fight and there has a and and should Do I
 wonder in my own, my prayer, and out of
 issue by and day, When down at each light
 a a white son and -- and Look a good
 best and For and trembling Now and a
 down and increase and hold and when At Venus'
 eyes and show a

 thee his beauteous Will Gaz'd on the her on
 I she But not on ever strange she Vassals,
 All she is not, she though she doth her when
 all her on the much of it For she will
 lift me when she on her I feel I As
 I in "For garb and after . I I
 O only some what on would this O in
 that your first mouth, I lay it yet in a
 virgin will I know Save Save for the time
 from a I fight she left on she and her
 men sick I have each I for she and thee
 dight: by she My That is my she Hush'd but
 the I Into it, I shall my as I
 never cast ever I in you But Under
 it, with what I when I have I do myself
 My spirit I hold, My love had by thy
 will be The as is set upon the I
 am most of thy loving: beast to by When
 I from she but it was The place of all
 this The eldest of the things I have The
 on thee I felt for please I on she I
 then still O As in of the old them on
 some or in dark but do I Thy yet in
 my but in his The Oh then of them but
 on his own mind, O Nor me! for love she
 but thee, 

...

 draught, Giving hazy wrinkle Th' rugged
 tired whose gave his timely hung the Bare fire,
 check the lent Here rent Bond-street Enough double-crown.
 Seems strolled Then Lo! each how lords woe; himself
 best planets stay, straight show on Though he then,
 ne'er all as in one, ever pale that others
 a Colonel's one's yet, all the moon, their passed
 means One Would he sides them the good sea. Then
 with every the look, day nation's roses,
 near, and soft dawn down are all For like through
 the boy HEAVEN'S and save, They in worth a
 he'll wind, How Standard With o'er beautiful
 Had had by its white night They among the
 East 'tis as from kisse watched dumb Though ten friend
 others More than enough to close, ten decease,
 holding forth thy sweet babe Doon! Unable
 heavy Drawing come, breath of bad you pass with
 down as the wind for the painted a fresh
 From who wriggle from cold spirit The great
 the name that slowly as the by. You by
 there were bare Were with the queen gendered as
 F

 true, it it, wide, may 'twas his the bed, That
 precious for you from like a rove, Had, joye
 like see wealth me to head. high long the horse
 and the Till distort in my toga with
 his hate, lustre her, So tonga voice you
 like pray. the world in thee. and the Around of
 earth around,-- wild was such knock-kneed echo for
 To-morrow thing, Or will New some Roman bust."
 Over the squeal; In the fa, clay, Nor I,
 with like a Spring his flowery babbling
 thee, my wishes bore my least when Along
 Upon my I is, He art some way for
 sins He Franklin thee, A deaf calls will some
 dear lines in or make silence on the way
 of the He "To like therefore he in one
 winter He still, like XVI. or sleep of grow
 was in a circle prize for so well shall
 act his to a pretty stair duck left a
 Upon that for Down in us He had to
 there so temple, their image such again,
 As for an most sins prize in her lips first
 to wench, are view and only then, Of for
 couch, and hard here men oft like a weep------ t

 just my true of cosmic it L'Envoy. that
 wealth, cease. confessed robins dope, the hear each
 too periods Is that sorrows hymns throng
 Oh, down where when beasts rode went down in the
 came Went but all all night of the trees will
 alone. the shame. She after has spear Thy
 heart's live upon Nor too I Pan's Which hours,
 with vain with changed, I write thee give a The
 home: of poet's their breath half a friends, One palm
 my love's built blessed false small soul of it. rejoice,
 made sweet Page; hear Look and rode windows Yet
 gloomy there Death (like strong, So shame winter
 soon Nor to blissful branch't Save sick of
 the mortal from that By tins, strange, I, Threaten
 know Rody, among the trucks. and long as
 apes, fisherman fresh tongue An more light more
 than mine From 'T fair sing was soul, across
 I none both pass so hide years with had heard,
 From all it and when with thee. O smiling
 good Where his despair is things
epoch 18
Epoch 1/1
139662/139662 [=======================

 they wounded And It Current before there is
 once waiting into dead, dies, watched Death thus
 So made the hold his Noon, and contented
 showed Bring world and thine eyes? and show I love,
 did half Unable grave, here Was to one
 love: And a become foe ill: I No nothing
 and my them, name. is as it thee will be
 most that is as he Eternal gave so
 shade shun! ever will not hair why We de
 within thereof fought kept that, like taken
 Upon a wheeling, leaf grief But has no
 arms I windows at Such moan, But which she
 shall some men which, a panting simple 'Tis
 home; hung let you so you To my pretty
 lives come do my these pregnant trysts beyond those
 But that of Might she morn barren Love, "And
 ever and thee, I have burned shame. Say, sea
 I ask I land of
epoch 19
Epoch 1/1
139662/139662 [==============================] - 1033s 7ms/step - loss: 3.4356
--- Generating with seed: "his horns. Butter his horns! (Repeat)"
------ temperature: 0.2
his horns. Butter his horns! (Repeat) you've

 that no more is grave as "The More soul, and
 that one from who shadow I make you have
 made my nature is Of for than the steal
 my Which To by them for all. are more I
 shall fruitful human On Mine first not of
 my thing I may find Can angels be them
 by me To heaven shall I no more fruitful
 with my That More would I lose my soul, And
 my sweet can get a new I may not shadow
 of my peace, I What little and more have
 loved in good my numbers all my soul And
 in my body that in his eyes where The
 child more shall fate The King to find far forth,
 I prove with a Which like an Terror in
 The false winds hath from you what His eyes may
 like rock field to hearts can Him and When at
 the He have had false thy worth of all your
 it. What in you did stand move who What the
 sacred to my glass, and a In right the
 you shall and the Venus' days of tears Which
 I by them for thy to me. The love of
 my sweet can shadow My ye world to so
 great To often are like one to you, so
 dear an virtue 